In [33]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# post process
cols = ["eyes","nose","nose_code","lips","lips_code"]
names = ['f1','f2','m1','m2']
for name in names:
    path = f'/content/{name}_face.csv'
    df = pd.read_csv(path)
    df_transposed_values = df.T

    # Reset index to make the transposed values into columns
    df_transposed = df_transposed_values.reset_index(drop=True)

    # Add the original column names as rows
    df_transposed.columns = cols
    df_transposed = df_transposed.reset_index(drop=True)
    print(len(df_transposed.columns))
    df_transposed.to_csv(f'/content/scl_{name}_face.csv')


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

sdf = pd.read_csv('/content/scl_m1_face.csv')
pdf = pd.read_csv('/content/py_m1_face.csv')

pdf

In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

chunk_levels = list(range(1, 6))  # Original chunk levels
x_labels = ["UNCROPPED"] + chunk_levels  # x-axis labels including 'full'

def plot_bar_with_annotations(df1, df2, image_name, erk):
    # Fetching data for the specified image name from both DataFrames
    if image_name in df1.columns and image_name in df2.columns:
        l1 = df1[image_name].values
        l2 = df2[image_name].values
    else:
        raise ValueError(f"Image name '{image_name}' not found in DataFrames.")

    # Number of bars (including 'full' as the first bar)
    N = len(x_labels)

    # Width of each bar
    bar_width = 0.25

    # Index values for each group of bars
    indices = np.arange(N)

    # Create the figure and axis
    fig, ax = plt.subplots(figsize=(10, 6))

    # Plotting the bars
    bar1 = ax.bar(indices - bar_width / 2, l1, bar_width, label='Imagehash')
    bar2 = ax.bar(indices + bar_width / 2, l2, bar_width, label='Scala')

    # Adding numbers on top of each bar
    for rect1, rect2 in zip(bar1, bar2):
        height1 = rect1.get_height()
        height2 = rect2.get_height()
        ax.annotate(f'{height1}', xy=(rect1.get_x() + rect1.get_width() / 2, height1),
                    xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')
        ax.annotate(f'{height2}', xy=(rect2.get_x() + rect2.get_width() / 2, height2),
                    xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')

    # Adding labels, title, and customizing the plot
    ax.set_xlabel('Chunking levels')
    ax.set_ylabel('Average hamming distance')
    strname = image_name
    if strname.find("code")!=-1:
        strname = "automatic " + strname.split('_')[0]
    if strname[-1]=='s':
        strname = strname[:-1]
    ax.set_title(f'Detection in {strname} modifications')

    # Set x-ticks and labels
    ax.set_xticks(indices)
    ax.set_xticklabels(x_labels)  # Include 'full' label

    ax.legend()

    # Save the plot to a file (PNG format)
    plt.tight_layout()
    plt.savefig(f'/content/drive/MyDrive/plots/avg_{image_name}.png')

    # Display the plot
    plt.show()

In [ ]:
import pandas as pd
import os

for t in ["f1","f2","m1","m2"]:
    sdf = pd.read_csv(f'/content/scl_{t}_face.csv')
    pdf = pd.read_csv(f'/content/py_{t}_face.csv')
    for name in pdf.columns:
        if name in pdf.columns and name in sdf.columns:
            os.makedirs(f'/content/drive/MyDrive/plots/{t}/',exist_ok=True)
            plot_bar_with_annotations(pdf,sdf,name,t)
        else:
            print(f"messed up with  {name}")
        print(name)

In [38]:
p_result = {}
s_result = {}
for features in ["eyes","nose","nose_code","lips","lips_code"]:
    arr_p = np.zeros(6)
    arr_s = np.zeros(6)
    cnt = 0
    for t in ["f1","f2","m1","m2"]:
        cnt += 1
        sdf = pd.read_csv(f'/content/scl_{t}_face.csv')
        pdf = pd.read_csv(f'/content/py_{t}_face.csv')
        arr_p += pdf[features]
        arr_s += sdf[features]
    arr_p /= cnt
    arr_s /= cnt

    p_result[features]=arr_p
    s_result[features]=arr_s

pdf = pd.DataFrame(p_result)
sdf = pd.DataFrame(s_result)

In [ ]:
for name in pdf.columns:
    plot_bar_with_annotations(pdf,sdf,name,-1)